# Mandatory Assignment 1: Counting Words

**This is the first of three mandatory assignments to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The assignment is to be done individually. You are under no circumstances allowed to collaborate with anyone on solving the exercises (cf. the full policy on this on the course website)**
- **You must hand in one Jupyter notebook (this notebook) with your solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you should use exactly the cells provided for your solution**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**

---
## Introduction
In this assignment you will build data structures for counting words in a text. Suppose you are given a very large corpus of text, and from time to time you need to count how many times a word occurs. You could write a program that searches the text from start to end every time you want to make a query, but for large texts, this will be very slow. A common way to handle this, is to preprocess the text into a data structure that contains exactly the information needed to answer specific queries like the frequency of a word.

Given a text, you should build data structures that can answer the following questions efficiently:
- How many times does a given word occur in the text? (exercise 2)
- How many times in total does a word starting with a given prefix occur? (exercise 3)

For each of these questions you should write a function that organizes data in a way (the data structure) that makes it possible to write an efficient function (the query) to answer the questions. A good data structure is one where the query is much faster than just searching the text while still is not using too much space.

You should not use any Python libraries (except `string`!) to solve the exercises. You may use build-ins like lists, dictionaries, `map`, `filter`, and so on.

You should provide implementations for the functions in this notebook. Do not change the names of the functions.

To test your programs, we will use the complete works of William Shakespeare. Please run the cells below to load the text and show a preview. You should be online before you do this. Note that a good solution will have to work for even larger texts than this.

In [1]:
import requests
text = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt').text

In [2]:
print('{} ...'.format(text[0:1000]))

This is the 100th Etext file presented by Project Gutenberg, and
is presented in cooperation with World Library, Inc., from their
Library of the Future and Shakespeare CDROMS.  Project Gutenberg
often releases Etexts that are NOT placed in the Public Domain!!

Shakespeare

*This Etext has certain copyright implications you should read!*

<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM
SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS
PROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE
WITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE
DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS
PERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED
COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY
SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>

*Project Gutenberg is proud to cooperate with The World Library*
in the presentation of The Complete Works of William Shakespeare
for your reading for educatio

---
## Exercise 1
In this exercise you should create a helper function for parsing a text into an iterable (e.g., a list) of words. You will need this in the subsequent exercises.

You should make sure that:
- each element of the iterable is exactly one word,
- all words are lower case,
- words do not contain punctuation.

Write a function `text_to_words()` that takes as input a string and output an iterable of the words in the string.

In [3]:
# You should implement this!
import string
#Here is an implementation not using the string package
def text_to_words_no_string(text):
        text = text.lower()
        words = ''
        for letter in text:
            if(letter == ' ' or letter.isalpha()):
                words += letter
        return words.split()
   # words = text.split()
#using string(probably more efficient)
def text_to_words(text):
    #First translate it removing all punctuations, then set all letters to lowercase, and finally return the splitted string
    return text.translate(str.maketrans('', '', string.punctuation)).lower().split()

    

In [4]:
# Should output 'this is a test'
' '.join(text_to_words('THIS is, a test!'))

'this is a test'

In [5]:
print(text_to_words('THIS is, a test!'))

['this', 'is', 'a', 'test']


One should note that string.punctuation does not have all signs, meaning that for example the Diaeresis symbol, e.g omlyd in Danish (¨ in ö) will not count as puntuations, simlar to  accent aigu (´) from french.

In [6]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [15]:
print(text_to_words(text[0:10000]))

['this', 'is', 'the', '100th', 'etext', 'file', 'presented', 'by', 'project', 'gutenberg', 'and', 'is', 'presented', 'in', 'cooperation', 'with', 'world', 'library', 'inc', 'from', 'their', 'library', 'of', 'the', 'future', 'and', 'shakespeare', 'cdroms', 'project', 'gutenberg', 'often', 'releases', 'etexts', 'that', 'are', 'not', 'placed', 'in', 'the', 'public', 'domain', 'shakespeare', 'this', 'etext', 'has', 'certain', 'copyright', 'implications', 'you', 'should', 'read', 'this', 'electronic', 'version', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'is', 'copyright', '19901993', 'by', 'world', 'library', 'inc', 'and', 'is', 'provided', 'by', 'project', 'gutenberg', 'etext', 'of', 'illinois', 'benedictine', 'college', 'with', 'permission', 'electronic', 'and', 'machine', 'readable', 'copies', 'may', 'be', 'distributed', 'so', 'long', 'as', 'such', 'copies', '1', 'are', 'for', 'your', 'or', 'others', 'personal', 'use', 'only', 'and', '2', 'are', 'not', 'distribute

---
## Exercise 2
In this exercise you should create a data structure that can give an answer to the following query.
- How many times does a given word occur in the text?

You should do the following:
- Write a function `build_word_count_data_structure()` that takes as input an iterable of strings, and outputs a data structure for looking up how many times a string occurs.

- Write a function `get_word_count()` that uses your data structure to count the number of times a word occurs.

- Explain with words how your data is organized in your data structure, how your function constructs it, and how a query works. Why is it efficient?

In [7]:
words_test = text_to_words('THIS is, a test! test')
# You should implement this!
def build_word_count_data_structure(words):
    dict = {}
    for word in words:
        if word in dict:
            dict[word] +=1
        else:
            dict[word] = 1
    return dict
        

In [8]:
ds = build_word_count_data_structure(words_test)
print(ds)

{'this': 1, 'is': 1, 'a': 1, 'test': 2}


In [9]:
# You should implement this!
def get_word_count(ds, word):
    if word in ds:
        return ds[word]
    else:
      return 0 

Allright. Let's try and use this for the romeo and juliet text above.

Do note that in this case we will have Romeo and Romeo's as two different words. The punctuation will remove the apostrophe s.t. Romeo -> romeo and Romeo's -> romeos 


In [30]:
ds = build_word_count_data_structure(text_to_words(text))
print("Number of \'Romeo\' in the text: "  + str(get_word_count(ds, 'romeo')))
print("Number of \'Romeo\'s\' in the text: " + str(get_word_count(ds, 'romeos')))

Number of 'Romeo' in the text: 137
Number of 'Romeo's' in the text: 19


### Explanation
_Explain your data structure with words_

The structure is very simple. It is basically a hash-map with a key and a value, since that is how dictonaries are implemented in python. The key is in this case the word and the value is the number of times it occurs.
The function constructs the datastructure by checking if the word has already been added, if it has the value is incremented by 1, and if not a new key is generated. When querying the function, we simply check if the word is even in the structure, and if it is the value can be accessed by just using the hashing function on the key. If the word is not in the structure a 0 is returned. The order of this operation is worst case $O(n)$ 

---
## Exercise 3
In this exercise you should create a data structure that can give an answer to the following query.
- How many times in total does a word starting with a given prefix occur?

For example, `bar` is a prefix of `bar`, `barracuda`, and `barrier`, and the result of the query should include the sum of the number of times each of those words occur.

You should do the following:
- Write a function `build_prefix_count_data_structure()` that takes as input a list of strings, and outputs a data structure for looking up how many times a prefix occurs in words.

- Write a function `get_prefix_count()` that uses your data structure to count the number of times a prefix occurs.

- Explain with words how your data is organized in your data structure, how your function constructs it, and how a query works. Why is it efficient?

In [11]:
# You should implement this!
class tree_node:
    def __init__(self,char):
        self.char = char
        self.children = []
        self.end = False
        self.nums = 0
def add_word(root,word):
    cur_node = root
    for char in word:
        #Using a for-else loop - if it does not break we go to the else section
         for child in cur_node.children:
                if(child.char == char):
                    child.nums += 1
                    cur_node = child
                    break;
         else:
            new_child = tree_node(char)
            new_child.nums += 1
            cur_node.children.append(new_child)
            cur_node = new_child
def build_prefix_count_data_structure(words):
    root = tree_node('')
    for word in words:
        add_word(root,word)
    return root
root = tree_node('')
add_word(root,'tiger')
add_word(root,'titan')
add_word(root,'total')

In [12]:
# You should implement this!
ds = build_prefix_count_data_structure(text_to_words('this is. a -test'))

In [13]:
def get_prefix_count(ds, prefix):
    node = ds
    #make sure it's lower case
    prefix = prefix.lower()
    #iterate over all chars
    for char in prefix:
        #iterate over all children
        for child in node.children:
            #we move to the next child if it exists
            if child.char == char:
                node = child
                break;
        else:
            #return 0 if the prefix does not exist
            return 0
    return node.nums

Now let us again apply this to the prefix "Romeo". This should give us the sum of all Romeo and Romeo's, e.g $137+19 =156$, the result from exercise 2.

In [35]:
ds = build_prefix_count_data_structure(text_to_words(text))
print("Number of words with the prefix \'Romeo\': " + str(get_prefix_count(ds,'romeo')))

Number of words with the prefix 'Romeo': 156


### Explanation
For this we are using a basic search tree. Suppose that we have the two words barrier and barracuda. Barrier is found by going through the tree as b->a->r->r->i->e->r meanwhile barracuda is at b->a->r->-r->a->c->u->d->a. At each of these nodes we then compute the number of entries we have here. If one really wanted to save memory, one could also just compute the number of entries by going through the tree from the prefix until the end and summing all the end-nodes up, although this would increase computation. 

It is efficient, because the query's have a very low order of complexity


One could improve the query time by making the mapping to children a hash-map instead, so one does not have to loop through all the children checking for characters. 

